<a href="https://colab.research.google.com/github/zxllxz2/disaster-dazzlers/blob/main/incident_join.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
credentials = {
    'region_name': 'us-east-1',
    'aws_access_key_id': 'xxx',
    'aws_secret_access_key': 'xxx',
    'aws_session_token': 'xxx'
}

In [ ]:
!pip install boto3
!pip install geopandas pyarrow

import geopandas as gd
import pyarrow.parquet as pq
import pandas as pd
import boto3, json

session = boto3.session.Session(**credentials)
s3 = session.client('s3')

In [18]:
session = boto3.session.Session(**credentials)
s3 = session.client('s3')

## Upload census tracts to S3

In [ ]:
import os

for root,dirs,files in os.walk('/content/census_tract'):
  for file in files:
    s3.upload_file(os.path.join(root[9:],file),'wwu-incidents', file)

## Join census tract, economics data, and incidents

In [ ]:
! unzip /content/census_tract.zip

In [48]:
df = gd.read_file('/content/census_tract/davidson/tl_2016_47_tract.shp')

In [53]:
df2 = pd.read_csv('/content/census_economic_data_all_tennessee_labels.csv')

In [50]:
incidents=pd.read_parquet('nfd_incidents_xd_seg.parquet')
incident_geo = gd.GeoDataFrame(
    incidents,
    geometry=gd.points_from_xy(incidents.longitude, incidents.latitude),
    crs=df.crs
)

In [51]:
jdi = gd.sjoin(df, incident_geo, how='inner', op='intersects')
len(jdi)

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


29768

In [ ]:
jdi = jdi.astype({'TRACTCE': 'int64', 'COUNTYFP': 'int64'})
jdi

In [ ]:
df2_clean = df2.astype({'tract': 'int64', 'county': 'int64'}).drop(['state'], axis=1)
df2_clean.head()

In [ ]:
df_comp = jdi.merge(df2_clean, how='left', left_on=['TRACTCE','COUNTYFP'], right_on=['tract','county'])
df_comp

In [59]:
df_comp.to_file('/content/incident_census_join_econ', driver="GeoJSON")

In [60]:
s3.upload_file(Bucket='wwu-incidents',Key='incident_census_join_econ',Filename='incident_census_join_econ')